In [177]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

### Rent & Buy Price Values

In [218]:
def process_rent(city, inputFile, outputFile):
    df_rent = pd.read_csv(inputFile)
    df_rent = df_rent[df_rent.Metro.str.contains(city, case=False, na=False)]
    df_rent = df_rent.rename(index=str, columns={'RegionName':'zipcode', 'City':'city','State':'state','Metro':'metro','CountyName':'county'})
    df_rent.reset_index(drop=True, inplace=True)
    df_rent.drop(['SizeRank'], axis=1, inplace=True)
    df_rent.to_csv(outputFile, index=False)

In [219]:
city = 'miami'
rent_files = [
    'datasets/Zip_MedianRentalPrice_1Bedroom.csv',
    'datasets/Zip_MedianRentalPricePerSqft_1Bedroom.csv',
    
    'datasets/Zip_MedianListingPrice_AllHomes.csv',
    'datasets/Zip_MedianListingPricePerSqft_AllHomes.csv',

    'datasets/Zip_MedianRentalPrice_2Bedroom.csv',
    'datasets/Zip_MedianRentalPricePerSqft_2Bedroom.csv',
    
    'datasets/Zip_MedianRentalPrice_SingleFamiliar.csv',
    'datasets/Zip_MedianRentalPricePerSqft_SingleFamiliar.csv',
    
    'datasets/Zip_PriceToRentRatio_AllHomes.csv'
]
output_rent_files = [
    'rent_1BR_' + city + '_total.csv',
    'rent_1BR_' + city + '_sqft.csv',

    'buy_home_' + city + '.csv',
    'buy_home_' + city + '_sqft.csv',
    
    'rent_2BR_' + city + '_total.csv',
    'rent_2BR_' + city + '_sqft.csv',
    
    'rent_SF_' + city + '_total.csv',
    'rent_SF_' + city + '_sqft.csv',
    
    'indicators_price_rent_ratio.csv'
]

### Indicators

In [220]:
city = 'miami'
indicator_files = [
    'datasets/Affordability_Income_2017Q4.csv',
    'datasets/Affordability_Wide_2017Q4_Public.csv',
    'datasets/BreakEven_PR_2018-032.csv',
    'datasets/home_forecast.csv',
    'datasets/rent_forecast.csv'
]
output_indicator_files = [
    'indicators_affordability_income_' + city + '.csv',
    'indicators_affordability_wide_' + city + '.csv',
    'indicators_breakeven_rate.csv',
    'indicators_home_forecast.csv',
    'indicators_rent_forecast.csv',
]

In [221]:
def process_indicators(city, inputFile, outputFile):
    df = pd.read_csv(inputFile)
    df = df[df.RegionName.str.contains(city, case=False, na=False)]
    df = df.rename(index=str, columns={'RegionID':'zipcode','RegionName':'city'})
    df.reset_index(drop=True, inplace=True)
    df.to_csv(outputFile, index=False)

In [222]:
def processDatasets():
    for i in range(0,len(rent_files)):
        process_rent(city, rent_files[i], output_rent_files[i])
    for i in range(0,len(indicator_files)):
        process_indicators(city, indicator_files[i], output_indicator_files[i])

In [223]:
processDatasets()

## Clean

In [262]:
rent_home_files = {
#     'rent_1BR_miami_sqft':'rent_1br_sqft',
    'rent_1BR_miami_total':'rent_1br_monthly',
    'rent_2BR_miami_sqft':'rent_2br_sqft',
    'rent_2BR_miami_total':'rent_2br_monthly',
    'rent_SF_miami_sqft':'rent_sf_sqft',
    'rent_SF_miami_total':'rent_sf_monthly',
    'buy_home_miami':'buy_home',
    'buy_home_miami_sqft':'buy_home_sqft'
}

In [240]:
df_rent1br_s.head(2)

,zipcode,city,state,metro,county,2010-02,2010-03,2010-04,2010-05,2010-06,...,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03
0,33160,Sunny Isles Beach,FL,Miami-Fort Lauderdale,Miami-Dade,NaN,NaN,NaN,NaN,NaN,...,1.935484,1.940135,2.013889,1.990050,1.870954,1.891074,1.847763,1.892116,1.890482,1.899758
1,33025,Miramar,FL,Miami-Fort Lauderdale,Broward,NaN,NaN,NaN,NaN,NaN,...,1.764706,1.805673,1.761038,1.789067,1.826972,1.737500,1.751011,1.794315,1.818889,1.889313


### Rent & Home Data Issues
1. Drop historical data, leave just last value
2. Add column to identify value (price_rent, price_rent_sqrt)
3. Combine values in single dataset

In [259]:
def cleanRentHomeData(file, indicator):
    df = pd.read_csv(file + '.csv')
    df_clean = df.iloc[:,:5]
    df_clean[indicator] = df.iloc[:,-1:]
    return df_clean
#     df_clean.to_csv(file + '.csv', index=False)

In [267]:
df_clean = cleanRentHomeData('rent_1BR_miami_sqft','rent_1br_sqft')
for k, v in rent_home_files.items():
    df_n = cleanRentHomeData(k,v)
    df_clean = df_clean.merge(df_n, how='outer',on=['zipcode','city','state','metro','county'])

In [273]:
df_clean.describe()

/Users/rhernande/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:4274: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,zipcode,rent_1br_sqft,rent_1br_monthly,rent_2br_sqft,rent_2br_monthly,rent_sf_sqft,rent_sf_monthly,buy_home,buy_home_sqft
count,176.000000,72.000000,76.000000,118.000000,119.000000,128.000000,115.000000,1.670000e+02,176.000000
mean,33470.732955,1.951816,1431.190789,1.657346,1796.764706,1.524956,2294.956522,4.026794e+05,234.769157
std,3103.291020,0.396685,305.964289,0.380903,531.843467,0.598452,593.186764,3.058902e+05,131.114387
min,33004.000000,1.314307,900.000000,1.142920,1200.000000,0.988024,1500.000000,6.795000e+04,62.418652
25%,33128.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166.736884
50%,33182.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196.793471
75%,33408.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256.968828
max,74354.000000,2.851711,2150.000000,2.898218,4200.000000,6.709120,4900.000000,3.200000e+06,1251.447757
